In [1]:
# чужие датасеты и фиксированное 512 

import os, glob
import sys
import json
from PIL import Image
from collections import Counter
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tifffile as tiff
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import cv2
import pandas as pd
from sklearn.model_selection import KFold

sys.path.append("/kaggle/input/detection-wheel")
import torch
torch.__version__  #torch 2.0
#!nvidia-smi   #CUDA Version: 11.4
! ls /usr/local

bin   cuda-11	 cuda-12    etc    include  man     sbin   src
cuda  cuda-11.8  cuda-12.1  games  lib	    nvidia  share


In [2]:

import sys
sys.path.append('../input/ensemble-boxes/')
from ensemble_boxes import weighted_boxes_fusion, weighted_masks_fusion

In [3]:
# # Install pycocotools package
import os

%mkdir work_dir_test
!pip install -qqq /kaggle/input/mmdet3-wheels-ando/addict-2.4.0-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdet3-wheels-ando/mmengine-0.7.3-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdet3-wheels-ando/mmcv-2.0.0-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/pycocotools-206/wheels/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/mmdet3-wheels-ando/terminaltables-3.1.10-py2.py3-none-any.whl
!pip install -qqq /kaggle/input/mmdet3-wheels-ando/mmdet-3.0.0-py3-none-any.whl

In [4]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib
from skimage.morphology import binary_dilation

def encode_binary_mask(mask: np.ndarray) -> t.Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    mask = mask.astype(bool)
    sg = mask.astype(np.uint8)
    #kernel = np.ones(shape=(2, 2), dtype=np.uint8)
    #mask = cv2.dilate(sg, kernel,3)
    #mask = binary_dilation(mask)
    #mask = binary_dilation(mask)
    #mask = binary_dilation(mask)
    # check input mask --
    if mask.dtype != bool:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

In [5]:
import os
import numpy as np
import torch
from PIL import Image
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [6]:
from mmdet.apis import init_detector, inference_detector

from mmengine.config import Config
from mmengine.runner import Runner


config_file = '/kaggle/input/hubmap-0707/HybridTaskCascade-353-1200/custom_config.py'
#config_file = '/kaggle/input/hubmap-gao/custom_config.py'

cfg = Config.fromfile(config_file )


checkpoint_file = '/kaggle/input/hubmap-0707/HybridTaskCascade-353-1200/best_coco_segm_mAP_epoch_11.pth'
checkpoint_file2 = '/kaggle/input/hubmap-0707/HybridTaskCascade-353-1200/best_coco_segm_mAP_epoch_11.pth'

model = init_detector(cfg, checkpoint_file, device=device)  # or device='cuda:0'
model2 = init_detector(cfg, checkpoint_file2, device=device)  # or device='cuda:0'

all_imgs = glob.glob('/kaggle/input/hubmap-hacking-the-human-vasculature/test/*.tif')
#all_imgs = glob.glob('/kaggle/input/hubmap-hacking-the-human-vasculature/train/*.tif')   # test for CUDA OOM error



Loads checkpoint by local backend from path: /kaggle/input/hubmap-0707/HybridTaskCascade-353-1200/best_coco_segm_mAP_epoch_11.pth
Loads checkpoint by local backend from path: /kaggle/input/hubmap-0707/HybridTaskCascade-353-1200/best_coco_segm_mAP_epoch_11.pth


In [7]:
ids = []
heights = []
widths = []
prediction_string = []

sample = None
import mmcv
confidence_thresholds = {0: 0.5, 1: 0.5, 2: 0.8}


for img in all_imgs:
    img_array = mmcv.imread(img,channel_order='rgb')
    [h, w, c] = img_array.shape 
    pred = inference_detector(model,img)
    
    pred2 = inference_detector(model2,img)
    
    pr_i = pred.pred_instances
    pr_i2 = pred2.pred_instances
    id = pred.img_path.split("/")[-1][:-4]
    shape = pred.img_shape
    height = h #shape[0]
    width = w #shape[1]

    scores = pr_i.scores.cpu().numpy()
    masks = pr_i.masks.cpu().numpy()
    boxes = pr_i.bboxes.cpu().numpy()

    scores2 = pr_i2.scores.cpu().numpy()
    masks2 = pr_i2.masks.cpu().numpy()
    boxes2 = pr_i2.bboxes.cpu().numpy()
    
    pred_strings = []

    # ensemble models using WMF. WMF is WBF applied to mask ensemble.
    MODEL_WEIGHTS = [1,1]
    wmf_inp_models = [0] * len(scores)
    wmf_inp_models += [1] * len(scores2)
    wmf_inp_models = np.array(wmf_inp_models)
    
    wmf_inp_pred_masks = np.vstack([masks, masks2])
    wmf_inp_pred_boxes = np.vstack([boxes, boxes2])
    wmf_inp_scores  = np.hstack([scores, scores2])
    
    
    wmf_masks, wmf_scores, wms_boxes = weighted_masks_fusion( wmf_inp_pred_masks, wmf_inp_pred_boxes, wmf_inp_scores, wmf_inp_models,
                                               skip_mask_thr=0,
                                               conf_type='model_weight',
                                               soft_weight=np.sum(MODEL_WEIGHTS),
                                               num_models=2,
                                               model_weights = MODEL_WEIGHTS)
    
    
    
    #for score, mask in zip(scores, masks):
    for score, mask in zip(wmf_scores, wmf_masks):

        pred_strings.append(" ".join(["0", str(score), encode_binary_mask(mask).decode()]))

        #print (score,counter)
        #plt.imshow(mask)
        #plt.show()
    ids.append(id)
    heights.append(height)
    widths.append(width)
    prediction_string.append(" ".join(pred_strings))

sub = pd.DataFrame({"id": ids, "height": heights, "width": widths, "prediction_string": prediction_string})
sub = sub.set_index("id")
sub.to_csv("submission.csv")



/opt/conda/lib/python3.10/site-packages/mmdet/models/roi_heads/mask_heads/fcn_mask_head.py:339: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  mask_preds = bboxes.new_tensor(mask_preds)


In [8]:
len(wmf_masks)

len(wmf_scores)

#masks.shape

100

In [9]:
''' 
if len(all_imgs)==1:  # visualisation
    #top20 = [sample[0]['masks'][i].cpu().numpy().reshape(512, 512) for i in range(min(20,len(sample[0]['masks'])))]
    top20 = masks[0:20,:,:]
    
    pred_img = np.zeros((512,512), dtype=np.float32)
    for i, j in enumerate(top20):
        pred_img += j * (1 - 1/len(top20)*i)
        pred_img = np.clip(pred_img, 0, 1)
        #print(sample[0]['scores'][i].cpu().numpy())
        plt.imshow(j)
        plt.show()
        
    plt.imshow(pred_img)
    plt.show()
'''

" \nif len(all_imgs)==1:  # visualisation\n    #top20 = [sample[0]['masks'][i].cpu().numpy().reshape(512, 512) for i in range(min(20,len(sample[0]['masks'])))]\n    top20 = masks[0:20,:,:]\n    \n    pred_img = np.zeros((512,512), dtype=np.float32)\n    for i, j in enumerate(top20):\n        pred_img += j * (1 - 1/len(top20)*i)\n        pred_img = np.clip(pred_img, 0, 1)\n        #print(sample[0]['scores'][i].cpu().numpy())\n        plt.imshow(j)\n        plt.show()\n        \n    plt.imshow(pred_img)\n    plt.show()\n"

In [10]:
sub

,height,width,prediction_string
id,,,
72e40acccadf,512,512,0 0.9773691296577454 eNoLzM82yTAItzfzMvI38Dc0A...
